In [2]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import re
from hanspell import spell_checker
import konlpy
from konlpy.tag import Okt
from keras_preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.layers import Dropout
import math
okt=Okt()
import cx_Oracle
import os
#한글 지원 방법
os.putenv("NLS_LANG",".UTF8")

### db에서 내용 소환

In [3]:
def makeDictFactory(cursor) :
    columnNames = [d[0] for d in cursor.description]
 
 
    def createRow(*args):
        return dict(zip(columnNames, args))
 
    return createRow




def sel_prof_sql(iD) : # select
    with cx_Oracle.connect("hr/hr@localhost:1521/xe") as conn:
        conn = cx_Oracle.connect('hr/hr@localhost:1521/xe')
        db = conn.cursor()
        sql = ''' 
            select * from member_profile
                where ID=(:n)
                
        '''
        db.execute(sql, n=iD)
        conn.commit()
        
        db.rowfactory = makeDictFactory(db)
        
        rows = db.fetchall()
        conn.close()
        return rows

def sel_info_sql(iD) : # select
    with cx_Oracle.connect("hr/hr@localhost:1521/xe") as conn:
        conn = cx_Oracle.connect('hr/hr@localhost:1521/xe')
        db = conn.cursor()
        sql = ''' 
            select * from member_info
                where ID=(:n)
                
        '''
        db.execute(sql, n=iD)
        conn.commit()
        
        db.rowfactory = makeDictFactory(db)
        
        rows = db.fetchall()
        conn.close()
        return rows

### 자기소개 점수 출력

In [170]:
from tensorflow.keras.models import load_model
text_model=load_model('./text_model-01-0.6416.hdf5') #여기에 모델명만 같게해주면 됨

In [162]:
def processing(text):
    spelled_sent = spell_checker.check(text)
    hanspell_sent = spelled_sent.checked 
    return hanspell_sent

def text_predict(sent):
    sentence_before=[]
    sentence_before.append(processing(sent))
    sent_txt=[]
    for sentence in sentence_before:
        temp_X=[]
        temp_X=okt.morphs(sentence)
        sent_txt.append(temp_X)
    
    
    
    max_words=38330
    tokenizer=Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(sent_txt)
    f_data=tokenizer.texts_to_sequences(sent_txt)
    max_len=130
    data_pad=pad_sequences(f_data,maxlen=max_len)
    
    import math
    
    score=text_model.predict(data_pad)[0][1]
    #result=math.log(score*100)*21.71472409516259
    #result_score=round(result,2)
    
    return score

In [91]:
def text_list(id):
    text=[]
    sentence=sel_prof_sql(id)[0]["INTRODUCE"]
    text.append(sentence)
    
    return text

In [92]:
text_list('testid3')

['안녕하세요 친하게 지내요']

In [89]:
Q="안녕하세요 잘부탁드립니다"

In [113]:
text_predict(text_list('testid3')[0]) # 출력결과는 float형태(숫자)

74.41

In [163]:
text_predict("ㅁ")

0.21142806

In [179]:
self="안녕하세요. 성격은 평범하고 깔끔하고 지적인걸 좋아합니다. 같이 공연 보러다니면서 맛있는거 먹으러 다니는걸 좋아합니다. 음식은 가리는건 딱히 없습니다. 상대분은 다정다감 하셨으면 좋겠습니다. 좋은 인연 만나고 싶습니다."

In [180]:
text_predict(self)

0.3923439

In [178]:
text_predict("안녕하세요. 성격은 평범하고 깔끔하고 지적인걸 좋아합니다. 같이 공연 보러다니면서 맛있는거 먹으러 다니는걸 좋아합니다. 음식은 가리는건 딱히 없습니다. 상대분은 다정다감 하셨으면 좋겠습니다. 좋은 인연 만나고 싶습니다.")

0.3923439

### 가족소개 점수 출력

In [7]:
family_model=load_model('./fam_text_model-01-0.6839.hdf5') #여기에 모델명만 같게해주면 됨

In [8]:
def family_predict(sent):
    sentence_before=[]
    sentence_before.append(sent)
    sent_txt=[]
    for sentence in sentence_before:
        temp_X=[]
        temp_X=okt.morphs(sentence)
        sent_txt.append(temp_X)
    
    
    
    max_words=19300
    tokenizer=Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(sent_txt)
    f_data=tokenizer.texts_to_sequences(sent_txt)
    max_len=130
    data_pad=pad_sequences(f_data,maxlen=max_len)
    
    import math
    
    p=family_model.predict(data_pad)[0][1]
    
    return p

In [9]:
F="부모님 두분과 같이 생활 하고 있습니다"

In [10]:
family_predict(F)# 출력결과는 float형태(숫자)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


0.4602557

### 프로필 점수 출력

In [132]:
import pickle
with open('profile_model.pkl','rb') as f: #모델명만 맞게 변환
    profile_model = pickle.load(f)

In [106]:
def make_list(id):
    data_info=sel_info_sql(id)
    data_profile=sel_prof_sql(id)
    
    list=[[data_profile[0]["LOCATION"],int(data_info[0]["BIRTH"][0:4]),
          int(data_info[0]["BIRTH"][4:6]),data_profile[0]["JOB"],
          data_profile[0]["SALARY"], data_profile[0]["HEIGHT"],
          data_profile[0]["WEIGHT"], data_profile[0]["BLOOD"],
          data_profile[0]["RELIGION"], data_profile[0]["EDUCATION"],
          data_profile[0]["PROPERTY"],
          data_profile[0]["PARENTS"],data_profile[0]["SMOKE"],
          data_profile[0]["DRINK"],data_profile[0]["MARRY_STATE"]]]
    
    return list

In [166]:
def profile_predict(list):   #입력값을 리스트로 묶어서 받음
    data=pd.read_csv("profile2.csv")
    df=pd.DataFrame(list,columns=["mem_loc","mem_birth_year",
                                 "mem_birth_month","mate_job",
                                 "mate_ann_salary","mate_height",
                                  "mate_weight","mate_blood","mate_religion",
                                 "mate_career","possess_property",
                                 "parents_slct","smoke_slct",
                                 "drink_slct","mate_slct"])
    data1=data.append(df,ignore_index=True)
    data1.reset_index(drop=True,inplace=True)
    categorical_features=['mem_loc','mate_job','mate_ann_salary','mate_blood',
                      'mate_religion','mate_career','possess_property',
                      'parents_slct','smoke_slct','drink_slct','mate_slct']
    for feature_name in categorical_features:
        one_hot=pd.get_dummies(data1[feature_name],prefix=feature_name) #원핫인코딩
        data1.drop(feature_name,axis=1,inplace=True) #기존 글자형태 컬럼 삭제
        data1=pd.concat([data1, one_hot],axis=1) #기존 데이터에 원핫데이터 붙이기
    
    p=profile_model.predict_proba([data1.iloc[104]])[0][1]
#     result=math.log(p*100)*21.71472409516259
#     result_score=round(result,2)
    
    
    return p

In [52]:
pd.read_csv("profile_data3.csv")

,mem_loc,mem_birth_year,mem_birth_month,mate_job,mate_ann_salary,mate_height,mate_weight,mate_blood,mate_religion,mate_career,wed_plan,possess_property,parents_slct,smoke_slct,drink_slct,mate_slct
0,강원,1910,1,경영,2천대,150,50,A,기타,"초,중졸",언제든지,3천미만,모두,Y,Y,Y
1,경기,1910,2,마케팅,3천대,160,60,B,기독교,고졸,6달,3천/7천,아버지만,N,N,N
2,경남,1910,3,무역,4천대,170,70,AB,불교,대학2졸,1년,7천/1억,어머니만,Y,Y,Y
3,경북,1910,4,유통,5천대,180,80,O,천주교,대학3졸,3달,1억/3억,작고,N,N,N
4,광주,1910,5,연구/개발,6천대,190,90,A,원불교,대학4졸,1달,3억/5억,고아,Y,Y,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,경북,1910,9,축산업,1억/3억,180,70,AB,이슬람교,대학원졸,3달,30억/50억,작고,Y,Y,Y
99,광주,1910,10,임업,2천미만,190,80,O,힌두교,대학원재학,1달,50억/100억,고아,N,N,N
100,대구,1910,11,광산업,3억/5억,150,90,B,도교,대학원중퇴,1년,100억/300억,어머니만,Y,Y,Y
101,대전,1910,12,빅데이터,5억/10억,160,100,AB,이슬람교,박사학위,3달,300억/500억,작고,N,N,N


In [168]:
test_list2=[["광주",1990,5,"유통/무역","3천만원대",178,80,"B형","무교","대학(4년)졸",
     "3천~7천 미만","두분 모두 계심","N","N","N"]]

In [164]:
test_list=[["서울",1980,1,"경영/사무","5천만원대",172,60,"B형","무교","대학(2,3년)졸",
     "1억~3억 미만","두분 모두 계심","N","N","N"]]

In [127]:
sel_info_sql('testid4')

[{'ID': 'testid4',
  'PW': 'pwtest4',
  'NAME': 'nametest2',
  'BIRTH': '19940502',
  'GENDER': 'm',
  'TEL': '010-1112-2423'}]

In [21]:
test_list

[['광주',
  1990,
  5,
  'it',
  '2천미만',
  178,
  80,
  'B',
  '무교',
  '대학4졸',
  '언제든지',
  '3천미만',
  '모두',
  'N',
  'N',
  'N']]

In [10]:
len(test_list[0])

16

In [142]:
profile_predict(test_list)  #결과 숫자로 반환

89.2

In [177]:
make_list('testid4')
print(make_list("testid4"),end=" ")

[['서울', 1994, 5, '유통/무역', '3천만원대', '176', '70', 'A형', '무교', '고졸', '3천~7천 미만', '두분 모두 계심', 'N', 'Y', 'N']] 

In [138]:
test_list

[['광주',
  1990,
  5,
  '유통/무역',
  '3천만원대',
  178,
  80,
  'B',
  '무교',
  '대학4졸',
  '3천~7천 미만',
  '두분 모두 계심',
  'N',
  'N',
  'N']]

In [154]:
profile_predict(make_list('testid4'))

95.49

In [165]:
profile_predict(test_list)

72.83

### 자기소개 글만 따로 빼오기

### 프로필 매력도(총 점수 계산)

In [29]:
def charming_score(sent,list):
    score1=text_predict(sent)
    score2=profile_predict(list)
    result=score1*0.5+score2*0.5
    score=math.log(result*100)*21.71472409516259
    result_score=round(score,2)
    
    return result_score

In [14]:
charming_score(Q,test_list)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


81.44

In [172]:
charming_score("ㅁ",test_list)

76.64

In [171]:
charming_score("안녕하세요",test_list2)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


85.04

In [70]:
sel_prof_sql('testid2')[0]["INTRODUCE"]

'안녕하세요 친하게 지내요'

In [100]:
make_list('testid4')

[['서울',
  1994,
  5,
  '유통/무역',
  '3천만원대',
  '176',
  '70',
  'A형',
  '무교',
  '고졸',
  '3천~7천 미만',
  '두분 모두 계심',
  'N',
  'Y',
  'N']]

In [20]:
len(make_list('testid2')[0])

16

In [75]:
charming_score('안녕하세요 친하게 지내요',test_list)

80.36

In [68]:
def profile_result(id):
    test=make_list(id)
    sentence=sel_prof_sql(id)[0]["INTRODUCE"]
    
    result=charming_score(sentence,test)
    
    return result

In [79]:
charming_score('안녕하세요 친하게 지내요',make_list('testid3'))

70.96

In [80]:
profile_result('testid3')

70.96

In [81]:
charming_score(sel_prof_sql('testid3')[0]["INTRODUCE"],make_list('testid3'))

70.96

### 관상

In [144]:
from collections import OrderedDict
import numpy as np
import cv2
import argparse
import dlib
import imutils
import math

In [157]:
facial_features_cordinates={}

FACIAL_LANDMARKS_INDEXES = OrderedDict([
    ("Mouth", (48, 68)),
    ("Right_Eyebrow", (17, 22)),
    ("Left_Eyebrow", (22, 27)),
    ("Right_Eye", (36, 42)),
    ("Left_Eye", (42, 48)),
    ("Nose", (27, 36)),
    ("Jaw", (0, 17))
])


def shape_to_numpy_array(shape, dtype="int"):
    

    # initialize the list of (x, y)-coordinates
    coordinates = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coordinates[i] = (shape.part(i).x, shape.part(i).y)

    

    # return the list of (x, y)-coordinates
    return coordinates

def visualize_facial_landmarks(image, shape, colors=None, alpha=0.75):

    

    # create two copies of the input image -- one for the
    # overlay and one for the final output image
    overlay = image.copy()
    output = image.copy()

    

    # if the colors list is None, initialize it with a unique

    # color for each facial landmark region
    if colors is None:
        colors = [(19, 199, 109), (79, 76, 240), (230, 159, 23),
                  (168, 100, 168), (158, 163, 32),
                  (163, 38, 32), (180, 42, 220)]

    

    # loop over the facial landmark regions individually
    for (i, name) in enumerate(FACIAL_LANDMARKS_INDEXES.keys()):
        

        # grab the (x, y)-coordinates associated with the
        # face landmark
        (j, k) = FACIAL_LANDMARKS_INDEXES[name]

        

        pts = shape[j:k]
        facial_features_cordinates[name] = pts

        

    # apply the transparent overlay
    cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)

    # return the output image
    #print(facial_features_cordinates)
    return output

def __angle_between(p1,p2): #두점 사이의 각도:(getAngle3P계산용) 시계방향으로 계산.P1-(0,0)
    ang1=np.arctan2(*p1[::-1])
    ang2=np.arctan2(*p2[::-1])
    res=np.rad2deg((ang1-ang2)%(2*np.pi))
    return res

def getAngle4P(p1, p2, p3, p4, direction="CW"): # 네점 사이의 각도 4-3 / 2-1: 1->2-> .... 4->3 으로 그려지는 각도
        pt1 = (p1[0] - p2[0], p1[1] - p2[1])              # 3 <- 4 <- V <-2<-1 으로 그려지는 각도
        pt2 = (p3[0] - p4[0], p3[1] - p4[1])        
        res = __angle_between(pt1, pt2)
        res = (res + 360) % 360
        if direction == "CCW":    #반시계방향
            res = (360 - res) % 360
        return res
    
def do_guansang(address):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    #이미지 입력(주소 받아서)
    image = cv2.imread(address)

    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    rects = detector(gray, 1)
    #얼굴에 특징 점 지정작업
    for (i, rect) in enumerate(rects):
    

    # determine the facial landmarks for the face region, then
    # convert the landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        shape = shape_to_numpy_array(shape)

    

        output = visualize_facial_landmarks(image, shape)
    
    #턱밑각 추출 
    jaw=facial_features_cordinates["Jaw"]
    
    point1=jaw[5]
    point2=jaw[6]
    point3=jaw[10]
    point4=jaw[11]
    
    face_angle=getAngle4P((point1[1],point1[0]),(point2[1],point2[0]),(point4[1],point4[0]),(point3[1],point3[0]),"CW")
    
    jaw_angle=(180-face_angle)/2 
    
    if jaw_angle<38:
        jaw_result="각진형"
    elif jaw_angle<43:
        jaw_result="역삼각형"
    elif jaw_angle<46:
        jaw_result="둥근형"
    else:
        jaw_result="각진형"
    
    #눈,코 비율
    right_Eye=facial_features_cordinates["Right_Eye"]
    left_Eye=facial_features_cordinates["Left_Eye"]
    nose=facial_features_cordinates["Nose"]
    
    left_h_rate=(jaw[8][1]-left_Eye[5][1])/(jaw[8][1]-left_Eye[2][1])
    left_w_rate=(jaw[8][0]-left_Eye[0][0])/(jaw[8][0]-left_Eye[3][0])
    right_h_rate=(jaw[8][1]-right_Eye[5][1])/(jaw[8][1]-right_Eye[2][1])
    right_w_rate=(right_Eye[3][0]-jaw[8][0])/(right_Eye[0][0]-jaw[8][0])
    nose_h_rate=(jaw[8][1]-nose[6][1])/(jaw[8][1]-nose[0][1])
    nose_w_rate=(nose[4][0]-jaw[0][0])/(nose[8][0]-jaw[0][0])
    
#     print(jaw_angle)
#     print(left_h_rate*left_w_rate)
#     print(right_h_rate*right_w_rate)
#     print(nose_h_rate*nose_w_rate)
    
    
    
#     if left_h_rate*left_w_rate<0.3804347826086956:
#         left_rate="LA"
#     elif left_h_rate*left_w_rate<0.41143423661599465:
#         left_rate="LB"
#     elif left_h_rate*left_w_rate<0.4394907407407408:
#         left_rate="LC"
#     else :
#         left_rate="LD"
        
#     if right_h_rate*right_w_rate<0.3975054020682204:
#         right_rate="RA"
#     elif right_h_rate*right_w_rate<0.4238095238095238:
#         right_rate="RB"
#     elif right_h_rate*right_w_rate<0.45063550673930786:
#         right_rate="RC"
#     else :
#         right_rate="RD"
    
    #눈 1사분면 기준:(0.3804347826086956+0.3975054020682204)/2, 0.38043478
    #눈 2사분면 기준:(0.41143423661599465+0.4238095238095238)/2, 0.41762185
    #눈 3사분면 기준:(0.4394907407407408+0.45063550673930786)/2, 0.4450631
    G_eye=(left_h_rate*left_w_rate+right_h_rate*right_w_rate)/2
    
    #결과는 G_eye*100로
    if G_eye<0.38043478:
        eye_degree="EA"
    elif G_eye<41762185:
        eye_degree="EB"
    elif G_eye<0.4450631:
        eye_degree="EC"
    else:
        eye_degree="ED"
    
    
#     if nose_h_rate*nose_w_rate<0.38380308536864305:
#         nose_rate="NA"
#     elif nose_h_rate*nose_w_rate<0.40012702445220705:
#         nose_rate="NB"
#     elif nose_h_rate*nose_w_rate<0.4150151153383838:
#         nose_rate="NC"
#     else:
#         nose_rate="ND"
    if nose_h_rate*nose_w_rate<0.38380308536864305:
        nose_degree="NA"
    elif nose_h_rate*nose_w_rate<0.40012702445220705:
        nose_degree="NB"
    elif nose_h_rate*nose_w_rate<0.4150151153383838:
        nose_degree="NC"
    else:
        nose_degree="ND"
    G_nose=nose_h_rate*nose_w_rate
    
    
    if left_Eye[3][1]>left_Eye[0][1]:
        eye_position="올라간 눈매"
    else:
        eye_position="내려간 눈매"
    
    #result=[jaw_result,left_rate,right_rate,nose_rate,eye_position]
    result=[jaw_result,G_eye,G_nose,eye_position,eye_degree,nose_degree]
    
    
    return result

def matching(adress1,adress2):
    
    result=[]
    result.append(do_guansang(adress1))
    result.append(do_guansang(adress2))
    
    return result

def g_count(man_adress1,female_adress2):
    result=matching(man_adress1,female_adress2)
    
    #턱
    if result[0][0]=="역삼각형" and result[1][0]=="둥근형":
        count1=100
    elif result[0][0]=="역삼각형" and result[1][0]=="각진형":
        count1=50
    elif result[0][0]=="각진형" and result[1][0]=="둥근형":
        count1=50
    elif result[0][0]=="각진형" and result[1][0]=="역삼각형":
        count1=50
    elif result[0][0]=="각진형" and result[1][0]=="각진형":
        count1=50
    elif result[0][0]=="둥근형" and result[1][0]=="각진형":
        count1=50
    elif result[0][0]=="둥근형" and result[1][0]=="역삼각형":
        count1=50
    else:
        count1=30
    
#     #왼눈 10,40,70,100
#     if result[0][1]=="LA" and result[1][1]=="LA":
#         count2=10
#     elif result[0][1]=="LA" and result[1][1]=="LB":
#         count2=40
#     elif result[0][1]=="LA" and result[1][1]=="LC":
#         count2=70
#     elif result[0][1]=="LA" and result[1][1]=="LD":
#         count2=100
#     elif result[0][1]=="LB" and result[1][1]=="LA":
#         count2=40
#     elif result[0][1]=="LB" and result[1][1]=="LB":
#         count2=10
#     elif result[0][1]=="LB" and result[1][1]=="LC":
#         count2=40
#     elif result[0][1]=="LB" and result[1][1]=="LD":
#         count2=70
#     elif result[0][1]=="LC" and result[1][1]=="LA":
#         count2=70
#     elif result[0][1]=="LC" and result[1][1]=="LB":
#         count2=40
#     elif result[0][1]=="LC" and result[1][1]=="LC":
#         count2=10
#     elif result[0][1]=="LC" and result[1][1]=="LD":
#         count2=40
#     elif result[0][1]=="LD" and result[1][1]=="LA":
#         count2=100
#     elif result[0][1]=="LD" and result[1][1]=="LB":
#         count2=70
#     elif result[0][1]=="LD" and result[1][1]=="LC":
#         count2=40
#     elif result[0][1]=="LD" and result[1][1]=="LD":
#         count2=10
    
#     #오눈
#     if result[0][2]=="RA" and result[1][2]=="RA":
#         count3=10
#     elif result[0][2]=="RA" and result[1][2]=="RB":
#         count3=40
#     elif result[0][2]=="RA" and result[1][2]=="RC":
#         count3=70
#     elif result[0][2]=="RA" and result[1][2]=="RD":
#         count3=100
#     elif result[0][2]=="RB" and result[1][2]=="RA":
#         count3=40
#     elif result[0][2]=="RB" and result[1][2]=="RB":
#         count3=10
#     elif result[0][2]=="RB" and result[1][2]=="RC":
#         count3=40
#     elif result[0][2]=="RB" and result[1][2]=="RD":
#         count3=70
#     elif result[0][2]=="RC" and result[1][2]=="RA":
#         count3=70
#     elif result[0][2]=="RC" and result[1][2]=="RB":
#         count3=40
#     elif result[0][2]=="RC" and result[1][2]=="RC":
#         count3=10
#     elif result[0][2]=="RC" and result[1][2]=="RD":
#         count3=40
#     elif result[0][2]=="RD" and result[1][2]=="RA":
#         count3=100
#     elif result[0][2]=="RD" and result[1][2]=="RB":
#         count3=70
#     elif result[0][2]=="RD" and result[1][2]=="RC":
#         count3=40
#     elif result[0][2]=="RD" and result[1][2]=="RD":
#         count3=10
    if result[0][4]=="EA" and result[1][4]=="EA":
        count2=10
    elif result[0][4]=="EA" and result[1][4]=="EB":
        count2=40
    elif result[0][4]=="EA" and result[1][4]=="EC":
        count2=70
    elif result[0][4]=="EA" and result[1][4]=="ED":
        count2=100
    elif result[0][4]=="EB" and result[1][4]=="EA":
        count2=40
    elif result[0][4]=="EB" and result[1][4]=="EB":
        count2=10
    elif result[0][4]=="EB" and result[1][4]=="EC":
        count2=40
    elif result[0][4]=="EB" and result[1][4]=="ED":
        count2=70
    elif result[0][4]=="EC" and result[1][4]=="EA":
        count2=70
    elif result[0][4]=="EC" and result[1][4]=="EB":
        count2=40
    elif result[0][4]=="EC" and result[1][4]=="EC":
        count2=10
    elif result[0][4]=="EC" and result[1][4]=="ED":
        count2=40
    elif result[0][4]=="ED" and result[1][4]=="EA":
        count2=100
    elif result[0][4]=="ED" and result[1][4]=="EB":
        count2=70
    elif result[0][4]=="ED" and result[1][4]=="EC":
        count2=40
    elif result[0][4]=="ED" and result[1][4]=="ED":
        count2=10



    #코
#     if result[0][3]=="NA" and result[1][3]=="NA":
#         count4=10
#     elif result[0][3]=="NA" and result[1][3]=="NB":
#         count4=40
#     elif result[0][3]=="NA" and result[1][3]=="NC":
#         count4=70
#     elif result[0][3]=="NA" and result[1][3]=="ND":
#         count4=100
#     elif result[0][3]=="NB" and result[1][3]=="NA":
#         count4=40
#     elif result[0][3]=="NB" and result[1][3]=="NB":
#         count4=10
#     elif result[0][3]=="NB" and result[1][3]=="NC":
#         count4=40
#     elif result[0][3]=="NB" and result[1][3]=="ND":
#         count4=70
#     elif result[0][3]=="NC" and result[1][3]=="NA":
#         count4=70
#     elif result[0][3]=="NC" and result[1][3]=="NB":
#         count4=40
#     elif result[0][3]=="NC" and result[1][3]=="NC":
#         count4=10
#     elif result[0][3]=="NC" and result[1][3]=="ND":
#         count4=40
#     elif result[0][3]=="ND" and result[1][3]=="NA":
#         count4=100
#     elif result[0][3]=="ND" and result[1][3]=="NB":
#         count4=70
#     elif result[0][3]=="ND" and result[1][3]=="NC":
#         count4=40
#     elif result[0][3]=="ND" and result[1][3]=="ND":
#         count4=10
    if result[0][5]=="NA" and result[1][5]=="NA":
        count3=10
    elif result[0][5]=="NA" and result[1][5]=="NB":
        count3=40
    elif result[0][5]=="NA" and result[1][5]=="NC":
        count3=70
    elif result[0][5]=="NA" and result[1][5]=="ND":
        count3=100
    elif result[0][5]=="NB" and result[1][5]=="NA":
        count3=40
    elif result[0][5]=="NB" and result[1][5]=="NB":
        count3=10
    elif result[0][5]=="NB" and result[1][5]=="NC":
        count3=40
    elif result[0][5]=="NB" and result[1][5]=="ND":
        count3=70
    elif result[0][5]=="NC" and result[1][5]=="NA":
        count3=70
    elif result[0][5]=="NC" and result[1][5]=="NB":
        count3=40
    elif result[0][5]=="NC" and result[1][5]=="NC":
        count3=10
    elif result[0][5]=="NC" and result[1][5]=="ND":
        count3=40
    elif result[0][5]=="ND" and result[1][5]=="NA":
        count3=100
    elif result[0][5]=="ND" and result[1][5]=="NB":
        count3=70
    elif result[0][5]=="ND" and result[1][5]=="NC":
        count3=40
    elif result[0][5]=="ND" and result[1][5]=="ND":
        count3=10
    
    
    #눈꼬리
    if result[0][3]=="올라간 눈매" and result[1][3]=="내려간 눈매":
        count4=100
    elif result[0][3]=="내려간 눈매" and result[1][3]=="올라간 눈매":
        count4=100
    else:
        count4=10
    
    #점수합산
    #before_result=count1*0.4+count2*0.125+count3*0.125+count4*0.25+count5*0.1
    before_result=count1*0.4+count2*0.25+count3*0.25+count4*0.1
    before_number=math.log(before_result)*21.71472409516259
    result_number=round(before_number)
    #순서 남자얼굴형,남자눈크기,남자코크기,남자눈매,여자얼굴형,여자눈크기,여자코크기,여자눈매,점수
    
    if result[0][4]=="EA":
        Eye_result_m="상위 25%"
    elif result[0][4]=="EB":
        Eye_result_m="상위 50%"
    elif result[0][4]=="EC":
        Eye_result_m="하위 50%"
    else:
        Eye_result_m="하위 25%"
        
    if result[1][4]=="EA":
        Eye_result_f="상위 25%"
    elif result[1][4]=="EB":
        Eye_result_f="상위 50%"
    elif result[1][4]=="EC":
        Eye_result_f="하위 50%"
    else:
        Eye_result_f="하위 25%"
    
    if result[0][5]=="EA":
        Nose_result_m="상위 25%"
    elif result[0][5]=="EB":
        Nose_result_m="상위 50%"
    elif result[0][5]=="EC":
        Nose_result_m="하위 50%"
    else:
        Nose_result_m="하위 25%"
        
    if result[1][5]=="EA":
        Nose_result_f="상위 25%"
    elif result[1][5]=="EB":
        Nose_result_f="상위 50%"
    elif result[1][5]=="EC":
        Nose_result_f="하위 50%"
    else:
        Nose_result_f="하위 25%"
    
    
    
    final_list=[result[0][0],Eye_result_m,Nose_result_m,result[0][3],result[1][0],Eye_result_f,Nose_result_m,result[1][3],result_number]
    
    return final_list

In [158]:
g_count("s1.png","s2.png")  #결과값 float(숫자)

['둥근형', '상위 25%', '하위 25%', '올라간 눈매', '역삼각형', '상위 25%', '하위 25%', '올라간 눈매', 71]

In [159]:
g_count("insung.jfif","mrim.jpg")

['역삼각형', '상위 50%', '하위 25%', '내려간 눈매', '각진형', '상위 50%', '하위 25%', '내려간 눈매', 84]

In [174]:
g_count("ys.jpg","seo.jpg")

['각진형', '상위 50%', '하위 25%', '내려간 눈매', '각진형', '상위 25%', '하위 25%', '내려간 눈매', 81]

In [175]:
g_count("ys.jpg","gain.jfif")

['각진형', '상위 50%', '하위 25%', '내려간 눈매', '둥근형', '상위 50%', '하위 25%', '내려간 눈매', 71]

In [20]:
self=text_predict(Q)
family=family_predict(F)
profile=profile_predict(test_list)
sum_num=self*0.4+family*0.4+profile*0.2

In [26]:
print(self)
print(family)
print(profile)

0.307799
0.4602557
0.5388518232359533


In [21]:
sum_num

0.41499225414304025

In [22]:
result_sum=math.log(sum_num)*21.71472409516259

In [23]:
result_sum

-19.09800046732663

In [31]:
result_sum=math.log(sum_num*100)*21.71472409516259

In [32]:
result_sum

80.90199953267337

In [27]:
sum_num1=self*0.2+family*0.2+profile*0.1

In [28]:
21.71472409516259/2

10.857362047581296

In [35]:
result_sum1=math.log(sum_num*100)

In [36]:
result_sum1

3.725674762346899

In [37]:
math.log(8+8+4)*21.71472409516259

65.05149978319906

In [38]:
math.log(4+4+2)*21.71472409516259

50.00000000000001

In [39]:
math.log(0)*21.71472409516259

ValueError: math domain error

In [ ]:
for i in range(len(not_merry["wed_plan"][i])):
    if not_merry["wed_plan"][i]=='a':
            not_merry["wed_plan"][i]="언제든지"
    elif not_merry["wed_plan"][i]=='6m':
            not_merry["wed_plan"][i]="6달"
    elif not_merry["wed_plan"][i]=='1y':
            not_merry["wed_plan"][i]="1년"
    elif not_merry["wed_plan"][i]=='3m':
            not_merry["wed_plan"][i]="3달"
    elif not_merry["wed_plan"][i]=='1m':
            not_merry["wed_plan"][i]="1달"
            
# plan(결혼계획)

In [ ]:
for i in range(len(not_merry["marry_state"][i])):
    if not_merry["marry_state"][i]=="w":
            not_merry["marry_state"][i]="N"
    elif not_merry["marry_state"][i]=='r':
            not_merry["marry_state"][i]="Y"

# marry_state(초혼인지 재혼인지)

In [ ]:
for i in range(len(not_merry["mem_loc"][i])):
    if not_merry["mem_loc"][i]=='a':
        not_merry["mem_loc"][i]="강원"
    elif not_merry["mem_loc"][i]=='b':
        not_merry["mem_loc"][i]="경기"
    elif not_merry["mem_loc"][i]=='c':
        not_merry["mem_loc"][i]="경남"
    elif not_merry["mem_loc"][i]=='d':
        not_merry["mem_loc"][i]="경북"
    elif not_merry["mem_loc"][i]=='e':
        not_merry["mem_loc"][i]="광주"
    elif not_merry["mem_loc"][i]=='f':
        not_merry["mem_loc"][i]="대구"
    elif not_merry["mem_loc"][i]=='g':
        not_merry["mem_loc"][i]="대전"
    elif not_merry["mem_loc"][i]=='h':
        not_merry["mem_loc"][i]="부산"
    elif not_merry["mem_loc"][i]=='i':
        not_merry["mem_loc"][i]="서울"
    elif not_merry["mem_loc"][i]=='z':
        not_merry["mem_loc"][i]="세종"
    elif not_merry["mem_loc"][i]=='j':
        not_merry["mem_loc"][i]="울산"
    elif not_merry["mem_loc"][i]=='k':
        not_merry["mem_loc"][i]="인천"
    elif not_merry["mem_loc"][i]=='l':
        not_merry["mem_loc"][i]="전남"
    elif not_merry["mem_loc"][i]=='n':
        not_merry["mem_loc"][i]="전북"
    elif not_merry["mem_loc"][i]=='m':
        not_merry["mem_loc"][i]="제주"
    elif not_merry["mem_loc"][i]=='o':
        not_merry["mem_loc"][i]="충남"
    elif not_merry["mem_loc"][i]=='p':
        not_merry["mem_loc"][i]="충북"
    elif not_merry["mem_loc"][i]=='q':
        not_merry["mem_loc"][i]="해외"
    elif not_merry["mem_loc"][i]=='y':
        not_merry["mem_loc"][i]="북한"

In [ ]:
for k in range(len(not_merry['mate_job'][i])):
    if not_merry["mate_job"][i]==0:
        not_merry["mate_job"][i]="기타"
    elif not_merry["mate_job"][i]==1:
        not_merry["mate_job"][i]="경영"
    elif not_merry["mate_job"][i]==2:
        not_merry["mate_job"][i]="마케팅"
    elif not_merry["mate_job"][i]==3:
        not_merry["mate_job"][i]="무역"
    elif not_merry["mate_job"][i]==4:
        not_merry["mate_job"][i]="유통"
    elif not_merry["mate_job"][i]==5:
        not_merry["mate_job"][i]="연구/개발"
    elif not_merry["mate_job"][i]==6:
        not_merry["mate_job"][i]="기계"
    elif not_merry["mate_job"][i]==7:
        not_merry["mate_job"][i]="전기"
    elif not_merry["mate_job"][i]==8:
        not_merry["mate_job"][i]="자동차"
    elif not_merry["mate_job"][i]==9:
        not_merry["mate_job"][i]="서비스"
    elif not_merry["mate_job"][i]==10:
        not_merry["mate_job"][i]="교육"
    elif not_merry["mate_job"][i]==11:
        not_merry["mate_job"][i]="금융"
    elif not_merry["mate_job"][i]==12:
        not_merry["mate_job"][i]="영업"
    elif not_merry["mate_job"][i]==13:
        not_merry["mate_job"][i]="문화/예술"
    elif not_merry["mate_job"][i]==14:
        not_merry["mate_job"][i]="방송"
    elif not_merry["mate_job"][i]==15:
        not_merry["mate_job"][i]="연극/영화"
    elif not_merry["mate_job"][i]==16:
        not_merry["mate_job"][i]="it"
    elif not_merry["mate_job"][i]==17:
        not_merry["mate_job"][i]="인터넷"
    elif not_merry["mate_job"][i]==18:
        not_merry["mate_job"][i]="디자인"
    elif not_merry["mate_job"][i]==19:
        not_merry["mate_job"][i]="건설"
    elif not_merry["mate_job"][i]==20:
        not_merry["mate_job"][i]="의료/보건"
    elif not_merry["mate_job"][i]==21:
        not_merry["mate_job"][i]="정부행정"
    elif not_merry["mate_job"][i]==22:
        not_merry["mate_job"][i]="농업"
    elif not_merry["mate_job"][i]==23:
        not_merry["mate_job"][i]="자영업"
    elif not_merry["mate_job"][i]==24:
        not_merry["mate_job"][i]="전문직/특수직"
    elif not_merry["mate_job"][i]==25:
        not_merry["mate_job"][i]="취업준비"
    elif not_merry["mate_job"][i]==26:
        not_merry["mate_job"][i]="창업준비"
    elif not_merry["mate_job"][i]==27:
        not_merry["mate_job"][i]="프리랜서"
    elif not_merry["mate_job"][i]==28:
        not_merry["mate_job"][i]="아르바이트"
    elif not_merry["mate_job"][i]==29:
        not_merry["mate_job"][i]="학생"
    elif not_merry["mate_job"][i]==30:
        not_merry["mate_job"][i]="법률"
    elif not_merry["mate_job"][i]==31:
        not_merry["mate_job"][i]="공기업/기관"
    elif not_merry["mate_job"][i]==32:
        not_merry["mate_job"][i]="군인/군무원"
    elif not_merry["mate_job"][i]==33:
        not_merry["mate_job"][i]="스포츠"
    elif not_merry["mate_job"][i]==34:
        not_merry["mate_job"][i]="사회복지"
    elif not_merry["mate_job"][i]==35:
        not_merry["mate_job"][i]="운송"
    elif not_merry["mate_job"][i]==36:
        not_merry["mate_job"][i]="물류/배송"
    elif not_merry["mate_job"][i]==37:
        not_merry["mate_job"][i]="설치"
    elif not_merry["mate_job"][i]==38:
        not_merry["mate_job"][i]="외국어/번역"
    elif not_merry["mate_job"][i]==39:
        not_merry["mate_job"][i]="보안/경비/경호"
    elif not_merry["mate_job"][i]==40:
        not_merry["mate_job"][i]="뷰티"
    elif not_merry["mate_job"][i]==41:
        not_merry["mate_job"][i]="외식"
    elif not_merry["mate_job"][i]==42:
        not_merry["mate_job"][i]="숙박"
    elif not_merry["mate_job"][i]==43:
        not_merry["mate_job"][i]="조선"
    elif not_merry["mate_job"][i]==44:
        not_merry["mate_job"][i]="판매"
    elif not_merry["mate_job"][i]==45:
        not_merry["mate_job"][i]="증권"
    elif not_merry["mate_job"][i]==46:
        not_merry["mate_job"][i]="회계/세무"
    elif not_merry["mate_job"][i]==47:
        not_merry["mate_job"][i]="고객상담"
    elif not_merry["mate_job"][i]==48:
        not_merry["mate_job"][i]="부동산/임대"
    elif not_merry["mate_job"][i]==49:
        not_merry["mate_job"][i]="신문/잡지"
    elif not_merry["mate_job"][i]==50:
        not_merry["mate_job"][i]="기획"
    elif not_merry["mate_job"][i]==51:
        not_merry["mate_job"][i]="사무/관리"
    elif not_merry["mate_job"][i]==52:
        not_merry["mate_job"][i]="홍보/광고"
    elif not_merry["mate_job"][i]==53:
        not_merry["mate_job"][i]="정비/as"
    elif not_merry["mate_job"][i]==54:
        not_merry["mate_job"][i]="미용/이용"
    elif not_merry["mate_job"][i]==55:
        not_merry["mate_job"][i]="예식"
    elif not_merry["mate_job"][i]==56:
        not_merry["mate_job"][i]="장례"
    elif not_merry["mate_job"][i]==57:
        not_merry["mate_job"][i]="프랜차이즈"
    elif not_merry["mate_job"][i]==58:
        not_merry["mate_job"][i]="여행"
    elif not_merry["mate_job"][i]==59:
        not_merry["mate_job"][i]="생산/제조"
    elif not_merry["mate_job"][i]==60:
        not_merry["mate_job"][i]="전자"
    elif not_merry["mate_job"][i]==61:
        not_merry["mate_job"][i]="가스/수도"
    elif not_merry["mate_job"][i]==62:
        not_merry["mate_job"][i]="금속/재료"
    elif not_merry["mate_job"][i]==63:
        not_merry["mate_job"][i]="섬유화학"
    elif not_merry["mate_job"][i]==64:
        not_merry["mate_job"][i]="석유화학"
    elif not_merry["mate_job"][i]==65:
        not_merry["mate_job"][i]="정유"
    elif not_merry["mate_job"][i]==66:
        not_merry["mate_job"][i]="식품공학"
    elif not_merry["mate_job"][i]==67:
        not_merry["mate_job"][i]="식품가공/제조"
    elif not_merry["mate_job"][i]==68:
        not_merry["mate_job"][i]="의류"
    elif not_merry["mate_job"][i]==69:
        not_merry["mate_job"][i]="유아/완구"
    elif not_merry["mate_job"][i]==70:
        not_merry["mate_job"][i]="투자"
    elif not_merry["mate_job"][i]==71:
        not_merry["mate_job"][i]="보험"
    elif not_merry["mate_job"][i]==72:
        not_merry["mate_job"][i]="공연"
    elif not_merry["mate_job"][i]==73:
        not_merry["mate_job"][i]="공예"
    elif not_merry["mate_job"][i]==74:
        not_merry["mate_job"][i]="도서/출판"
    elif not_merry["mate_job"][i]==75:
        not_merry["mate_job"][i]="사진"
    elif not_merry["mate_job"][i]==76:
        not_merry["mate_job"][i]="인쇄/인화"
    elif not_merry["mate_job"][i]==77:
        not_merry["mate_job"][i]="음악/악기"
    elif not_merry["mate_job"][i]==78:
        not_merry["mate_job"][i]="엔터테인먼트"
    elif not_merry["mate_job"][i]==79:
        not_merry["mate_job"][i]="게임"
    elif not_merry["mate_job"][i]==80:
        not_merry["mate_job"][i]="만화/웹툰"
    elif not_merry["mate_job"][i]==81:
        not_merry["mate_job"][i]="통신"
    elif not_merry["mate_job"][i]==82:
        not_merry["mate_job"][i]="ict"
    elif not_merry["mate_job"][i]==83:
        not_merry["mate_job"][i]="반도체"
    elif not_merry["mate_job"][i]==84:
        not_merry["mate_job"][i]="led/광산업"
    elif not_merry["mate_job"][i]==85:
        not_merry["mate_job"][i]="환경"
    elif not_merry["mate_job"][i]==86:
        not_merry["mate_job"][i]="모바일"
    elif not_merry["mate_job"][i]==87:
        not_merry["mate_job"][i]="로봇"
    elif not_merry["mate_job"][i]==88:
        not_merry["mate_job"][i]="나노"
    elif not_merry["mate_job"][i]==89:
        not_merry["mate_job"][i]="바이오"
    elif not_merry["mate_job"][i]==90:
        not_merry["mate_job"][i]="신소재"
    elif not_merry["mate_job"][i]==91:
        not_merry["mate_job"][i]="신재생에너지"
    elif not_merry["mate_job"][i]==92:
        not_merry["mate_job"][i]="우주/항공"
    elif not_merry["mate_job"][i]==93:
        not_merry["mate_job"][i]="인테리어"
    elif not_merry["mate_job"][i]==94:
        not_merry["mate_job"][i]="토목"
    elif not_merry["mate_job"][i]==95:
        not_merry["mate_job"][i]="설계"
    elif not_merry["mate_job"][i]==96:
        not_merry["mate_job"][i]="제약"
    elif not_merry["mate_job"][i]==97:
        not_merry["mate_job"][i]="종교"
    elif not_merry["mate_job"][i]==98:
        not_merry["mate_job"][i]="어업"
    elif not_merry["mate_job"][i]==99:
        not_merry["mate_job"][i]="축산업"
    elif not_merry["mate_job"][i]==100:
        not_merry["mate_job"][i]="임업"
    elif not_merry["mate_job"][i]==101:
        not_merry["mate_job"][i]="광산업"
    elif not_merry["mate_job"][i]==102:
        not_merry["mate_job"][i]="빅데이터"
    elif not_merry["mate_job"][i]==103:
        not_merry["mate_job"][i]="낙농업"
        
# job(직업)

In [ ]:
for k in range(len(not_merry['salary'][i])):
    if not_merry["salary"][i]==2:
        not_merry["salary"][i]="2천대"
    elif not_merry["salary"][i]==3:
        not_merry["salary"][i]="3천대"
    elif not_merry["salary"][i]==4:
        not_merry["salary"][i]="4천대"
    elif not_merry["salary"][i]==5:
        not_merry["salary"][i]="5천대"
    elif not_merry["salary"][i]==6:
        not_merry["salary"][i]="6천대"
    elif not_merry["salary"][i]==7:
        not_merry["salary"][i]="7천대"
    elif not_merry["salary"][i]==8:
        not_merry["salary"][i]="8천대"
    elif not_merry["salary"][i]==9:
        not_merry["salary"][i]="9천대"
    elif not_merry["salary"][i]==10:
        not_merry["salary"][i]="1억/3억"
    elif not_merry["salary"][i]==12:
        not_merry["salary"][i]="2천미만"
    elif not_merry["salary"][i]==13:
        not_merry["salary"][i]="3억/5억"
    elif not_merry["salary"][i]==14:
        not_merry["salary"][i]="5억/10억"
    elif not_merry["salary"][i]==15:
        not_merry["salary"][i]="10억/20억"
    elif not_merry["salary"][i]==16:
        not_merry["salary"][i]="30억이상"
    elif not_merry["salary"][i]==18:
        not_merry["salary"][i]="20억/30억"
        
#salary (연봉)

In [ ]:
for i in range(len(not_merry["property"][i])):
    if not_merry["property"][i]=='b':
        not_merry["property"][i]="3천미만"
    elif not_merry["property"][i]=='c':
        not_merry["property"][i]="3천/7천"
    elif not_merry["property"][i]=='d':
        not_merry["property"][i]="7천/1억"
    elif not_merry["property"][i]=='e':
        not_merry["property"][i]="1억/3억"
    elif not_merry["property"][i]=='f':
        not_merry["property"][i]="3억/5억"
    elif not_merry["property"][i]=='g':
        not_merry["property"][i]="5억/10억"
    elif not_merry["property"][i]=='h':
        not_merry["property"][i]="10억/20억"
    elif not_merry["property"][i]=='i':
        not_merry["property"][i]="30억/50억"
    elif not_merry["property"][i]=='j':
        not_merry["property"][i]="50억/100억"
    elif not_merry["property"][i]=='k':
        not_merry["property"][i]="100억/300억"
    elif not_merry["property"][i]=='l':
        not_merry["property"][i]="300억/500억"
    elif not_merry["property"][i]=='n':
        not_merry["property"][i]="500억이상"
    elif not_merry["property"][i]=='m':
        not_merry["property"][i]="20억/30억"
        
#property(보유재산)

In [ ]:
for i in range(len(not_merry["parents"][i])):
    if not_merry["parents"][i]=='a':
        not_merry["parents"][i]="모두"
    elif not_merry["parents"][i]=='b':
        not_merry["parents"][i]="아버지만"
    elif not_merry["parents"][i]=='c':
        not_merry["parents"][i]="어머니만"
    elif not_merry["parents"][i]=='d':
        not_merry["parents"][i]="작고"
    elif not_merry["parents"][i]=='z':
        not_merry["parents"][i]="고아"
#부모님